# Process Raw News Data About Nvidia

This notebook processes raw JSON news data about Apple and related tech topics from the Webarticles. It:
1. Loads the raw JSON data
2. Normalizes and cleans the data into a pandas DataFrame
3. Exports the processed data to CSV and Excel formats

## Import Required Libraries

In [61]:
import pandas as pd
import os
from pathlib import Path

## Configuration

Define paths and settings to make the notebook more maintainable and configurable.

In [62]:
INPUT_PATH = Path('../data/raw/news')
OUTPUT_DIR = Path('../data/processed/news')
CSV_OUTPUT = OUTPUT_DIR / 'news_articles.csv'
EXCEL_OUTPUT = OUTPUT_DIR / 'news_articles.xlsx'

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

SELECTED_COLUMNS = ['source.name', 'author', 'title', 'description', 'content', 'url']

## Data Loading and Processing

Load the raw JSON news data and convert it to a structured DataFrame.

In [63]:
def load_all_articles(input_dir):
    """Load all news articles from JSON files in the specified directory."""
    all_articles = []
    
    # Check if directory exists
    if not input_dir.exists() or not input_dir.is_dir():
        print(f"Warning: Input directory {input_dir} does not exist or is not a directory")
        return all_articles
    
    # Process each JSON file
    for file_path in input_dir.glob('*.json'):
        try:
            raw_data = pd.read_json(file_path)
            articles = raw_data.get('articles', [])
            all_articles.extend(articles)
        except Exception as e:
            print(f"Error loading {file_path.name}: {e}")
    
    return all_articles

# Load and process
articles = load_all_articles(INPUT_PATH)
if articles:
    # Convert to DataFrame and normalize nested JSON
    df_news = pd.json_normalize(articles)
    
    # Check if all requested columns exist
    missing_columns = [col for col in SELECTED_COLUMNS if col not in df_news.columns]
    if missing_columns:
        print(f"Warning: Missing columns in data: {missing_columns}")
        available_columns = [col for col in SELECTED_COLUMNS if col in df_news.columns]
        df_selected = df_news[available_columns] if available_columns else df_news
    else:
        df_selected = df_news[SELECTED_COLUMNS]
    
    # Report source distribution if that column exists
    if 'source.name' in df_selected.columns:
        source_counts = df_selected['source.name'].value_counts()
        sources_str = ', '.join(source_counts.index)
        print(f"Loaded {len(df_selected)} articles from: {sources_str}.")
    else:
        print(f"Loaded {len(df_selected)} articles.")
else:
    print("No articles found.")

Loaded 79 articles from: MacRumors, The Verge, Gizmodo.com, Wired, Android Central, Business Insider, CNET.


## Preview the Processed Data

In [64]:
df_selected

,source.name,author,title,description,content,url
0,The Verge,"Tom Warren, Jess Weatherbed",The EU isn’t happy with Apple’s tax on alterna...,The European Commission has just issued its fi...,The European Commission has also closed its in...,https://www.theverge.com/news/636196/apple-eu-...
1,Wired,Adrienne So,The Apple Watch Turns 10. Here's How Far It's ...,"When the Apple Watch launched, it was unclear ...","Every year, Apple launches one standout health...",https://www.wired.com/story/apple-watch-turns-10/
2,The Verge,Chris Welch,Apple stumbles with latest AirPods Max firmware,"Last week, Apple announced that lossless audio...",Lossless audio and ultra low latency are still...,https://www.theverge.com/news/642140/apple-air...
3,Wired,Brenda Stolyar,"22 Best MacBook Accessories (2025), Tested and...","From charging adapters to external monitors, w...",More Good Accessories\r\nApple 35W Dual USB-C ...,https://www.wired.com/gallery/best-macbook-acc...
4,The Verge,Todd Haselton,Apple has its biggest stock drop in five years...,"Shares of Apple, Amazon, and other tech stocks...",Amazon and Meta shares are down about 7 percen...,https://www.theverge.com/news/642598/apple-sto...
...,...,...,...,...,...,...
74,MacRumors,Juli Clover,Apple's Secret Robotics Team Gets New Leadership,Apple is removing another project from AI chie...,Apple is removing another project from AI chie...,https://www.macrumors.com/2025/04/24/apple-rob...
75,MacRumors,Tim Hardwick,"How Apple Could Navigate Trump's Tariffs, Acco...",Apple is likely to take a multi-pronged approa...,Apple is likely to take a multi-pronged approa...,https://www.macrumors.com/2025/04/07/how-apple...
76,MacRumors,Joe Rossignol,Apple Hit With More Class Action Lawsuits Over...,Apple has been hit with at least two more prop...,Apple has been hit with at least two more prop...,https://www.macrumors.com/2025/04/10/apple-hit...
77,CNET,Zachary McAuliffe,Download iOS 18.4.1 Now to Fix These Vulnerabi...,Apple said these vulnerabilities may've been e...,"Apple released iOS 18.4.1 on April 16, more th...",https://www.cnet.com/tech/services-and-softwar...


## Export Processed Data

Save the processed data to CSV and Excel formats for further analysis.

In [57]:
def export_data(df, csv_path, excel_path):
    """Export dataframe to CSV and Excel formats."""
    try:
        # Export to CSV
        df.to_csv(csv_path, index=False, encoding='utf-8')
        
        # Export to Excel
        df.to_excel(excel_path, index=False)
        
        return True
    except Exception as e:
        print(f"Error exporting data: {e}")
        return False

# Export the data
if export_data(df_selected, CSV_OUTPUT, EXCEL_OUTPUT):
    print(f"Data successfully exported to:")
    print(f"- CSV: {CSV_OUTPUT}")
    print(f"- Excel: {EXCEL_OUTPUT}")

Data successfully exported to:
- CSV: ../data/processed/news/news_articles.csv
- Excel: ../data/processed/news/news_articles.xlsx


## Conclusion

This notebook has successfully:
1. Loaded raw news data from a JSON file
2. Processed and normalized the data into a structured format
3. Performed basic analysis on the dataset
4. Exported the processed data to CSV and Excel formats for further use

The processed data includes 10 TechCrunch articles from April 2025, with 2 directly related to Nvidia. The data can now be used for further analysis, sentiment analysis, or as part of a larger dataset for tech news tracking.